In [1]:
using JuMP
using CPLEX
using CPUTime
using Graphs

In [2]:
struct Clients
    nb_clients
    X
    Y
    h
    L
    L0
    d
end

In [3]:
function Read_PRP_instance(filename)
    hashmap = Dict()
    open(filename) do f
        
        demands = false
        listing_active = false
        current_value = 0
        
        nbc = 0
        X = Array{Float64}(undef, 0)
        Y = Array{Float64}(undef, 0)
        h = Array{Float64}(undef, 0)
        L = Array{Float64}(undef, 0)
        L0 = Array{Float64}(undef, 0)
        d = Array{Array{Float64}}(undef, 0)
        
        for (i, line) in enumerate(eachline(f))
            x = split(line, " ")
            deleteat!(x, findall(e->e=="", x))
            current_item = x[1]
            
            if(current_item == "0" || listing_active) # Liste de clients / demandes
                listing_active = true
                if(current_item == "d")
                    demands = true
                elseif(demands) # Enregistrement des demandes
                    push!(d, parse.(Float64, x)[2:end])
                else # enregistrement des clients
                    nbc += 1
                    push!(X, parse(Float64, x[2]))
                    push!(Y, parse(Float64, x[3]))
                    push!(h, parse(Float64, x[6]))
                    push!(L, parse(Float64, x[8]))
                    push!(L0, parse(Float64, x[10]))
                end
            else # variables générales du problème
                try
                    current_value = parse(Int64, x[2])
                catch err
                    current_value = 10000000000
                end
                hashmap[current_item] = current_value
            end
        end
        C = Clients(nbc, X, Y, h, L, L0, d)
        hashmap["Clients"] = C
    end 
    return hashmap
end

Read_PRP_instance (generic function with 1 method)

In [4]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 10000000000
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [143.0, 89.0, 76.0, 285.0, 401.0, 16.0, 267.0, 249.0…
  "n"       => 14

In [5]:
h["Clients"].h[14]

6.0

In [6]:
function dist(C, i, j, t, mc)
    if(t == 1)
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) + 0.5
    else
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) * mc
    end
end

dist (generic function with 1 method)

In [7]:
function calcul_dist(p)
    if(p["Type"] == 1)
        mc = 0
    else
        mc = p["mc"]
    end
    c = Array{Float64}(undef, (p["n"]+1, p["n"]+1))
    for i in 1:p["n"]+1
        for j in 1:p["n"]+1            
            c[i, j] = dist(p["Clients"], i, j, p["Type"], mc)
        end
    end
    return c
end

calcul_dist (generic function with 1 method)

In [8]:
sum(h["Clients"].h[i] for i in 1:15)

106.0

In [9]:
sum(sum(h["Clients"].d[jj][tt] for jj in 1:h["Clients"].nb_clients-1) for tt in 1:h["l"])

1380.0

In [10]:
for i in 1:6
    print(i)
end

123456

In [11]:
h["Clients"].nb_clients

15

In [12]:
a = [1, 2]
b = [1,2,3]
c = b[b .∉ Ref(a)]
print(c)

[3]

In [13]:
function partiesliste(seq, min_size, max_size)
    p = [] #Array{Float64}(undef, 2^size(seq)[1]-1 - 2^min_size)
    i = 1
    imax = 2 ^ size(seq)[1]
    while(i <= imax)
        s = []
        j, jmax = 0, size(seq)[1]-1
        while(j <= jmax)
            if((i>>j)&1 == 1)
                append!(s, seq[j+1])
            end
            j += 1
        end
        if(size(s)[1] >= min_size && size(s)[1] <= max_size)
            append!(p, [s])
        end
        i += 1
    end
    return p
end

partiesliste (generic function with 1 method)

In [14]:
function Resolve_PRP(p)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Bin)  # Quantité produite
    @variable(LP, inv[1:n, 1:l], Int)  # Inventaire des clients
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int) # qt délivré au client i à la période t
    @variable(LP, z[1:n, 1:l], Int) # 1 client i visité au temps t, 0 sinon (Attention z[0, t] nb de véhicules 
                                    # qui quittent l'usine au temps t)
    @variable(LP, w[1:n, 1:l], Int) # chargement véhicule avant de faire une livraison au client i au temps t 

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n if i ≠ j) for j in 1:n) for t in 1:l))
    
    indices_array = [i for i in 1:n-1]
    allS = partiesliste(indices_array, 1, n-2)
    
    for t in 1:l # Pour toutes les périodes (l=nb de prériodes)
        sum_d_current_t = sum(sum(p["Clients"].d[ii][tt] for ii in 1:n-1) for tt in t:l)
        Mt = min(p["C"], sum_d_current_t)
        @constraint(LP, prod[t] <= Mt * y[t]) # (4) OK
        @constraint(LP, inv[1, t] <= p["Clients"].L[1]) # (5) OK
        @constraint(LP, z[1, t] <= p["k"]) # (10) OK | Attention k == m sur l'article
        @constraint(LP, 0 <= prod[t]) # (13.1) OK
        @constraint(LP, 0 <= y[t] <= 1) # (14.1) OK
        # Contrainte 16 définie par le Int lors de la déclaration de la variable
        
        
        for ii in allS
            jj = indices_array[indices_array .∉ Ref(ii)]
            # (17) OK :
            @constraint(LP, sum(sum(x[i, j, t] for j in jj) for i in ii) >= sum(q[i, t] for i in ii) / p["Q"])
            if(size(ii)[1] >= 2 && size(ii)[1] <= n-3)
                @constraint(LP, sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    size(ii)[1] - sum(q[i, t] for i in ii) / p["Q"]) # (18) OK
                @constraint(LP, p["Q"] * sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    sum(p["Q"] * z[i, t] - q[i, t] for i in ii)) # (19) OK
            end
        end
        
        if(t ≠ 1) 
            @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 1:n) + inv[1, t]) # (2) OK
        end
        for i in 1:n # Parcours de tous les clients
            Mit = min(p["Clients"].L[i], p["Q"], sum_d_current_t)
            xijt_sum = sum(x[i, jj, t] for jj in 1:n)
            if(i ≠ 1) # Nc
                if(t ≠ 1)
                    @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i-1][t] + inv[i, t]) # (3) OK
                    @constraint(LP, inv[i, t-1] + q[i, t] <= p["Clients"].L[i]) # (6) OK
                end
                @constraint(LP, q[i, t] <= Mit * z[i, t]) # (7) OK
                @constraint(LP, xijt_sum == z[i, t]) # (8) OK
                @constraint(LP, 0 <= w[i, t]) # (12.1) OK
                @constraint(LP, w[i, t] <= p["Q"] * z[i, t]) # (12.2) OK
                @constraint(LP, 0 <= z[i, t] <= 1)
            end
            @constraint(LP, xijt_sum + sum(x[jj, i, t] for jj in 1:n) == 2 * z[i, t]) # (9) OK
            @constraint(LP, 0 <= inv[i, t]) # (13.2) OK
            @constraint(LP, 0 <= q[i, t]) # (13.3) OK 
            
            for j in 1:n
                if(i ≠ j)
                    @constraint(LP, w[i, t] - w[j, t] <= q[i, t] - Mit * (1 - x[i, j, t])) # (11) OK
                end
                @constraint(LP, 0 <= x[i, j, t] <= 1) # (14.2) OK
            end
        end
    end
    
    # print(LP)
    # println()

    optimize!(LP)
   
    println(solution_summary(LP, verbose=true))
    
    return 0
end

Resolve_PRP (generic function with 1 method)

In [15]:
Resolve_PRP(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Presolve has eliminated 1741 rows and 1442 columns...
Presolve has improved bounds 823 times...
Tried aggregator 2 times.
MIP Presolve eliminated 1741 rows and 1442 columns.
MIP Presolve modified 363479 coefficients.
Aggregator did 6 substitutions.
Reduced MIP has 296268 rows, 1714 columns, and 17521134 nonzeros.
Reduced MIP has 1439 binaries, 275 generals, 0 SOSs, and 0 indicators.
Presolve time = 25.76 sec. (19502.08 ticks)
Probing fixed 78 vars, tightened 0 bounds.
Probing time = 1.51 sec. (965.40 ticks)
* Solver : CPLEX

* Status
  Termination status : INFEASIBLE
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Result count       : 0
  Has duals          : false
  Message from the solver:
  "integer infeasible"

* Candidate solution
  Objective bound      : 1.00000e+75

* Work counters
  Solve time (sec)   : 4.72139e+01
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : 0


Root node 

0

In [ ]:
function Resolve_PRP2(p)
    LP = Model(CPLEX.Optimizer)
    
    c = calcul_dist(p)
    
    n = p["Clients"].nb_clients      # Nombre de clients (usine inclue)
    l = p["l"]                       # Nombre de périodes
    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Bin)  # Quantité produite
    @variable(LP, inv[1:n, 1:l], Int)  # Inventaire des clients
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int) # qt délivré au client i à la période t
    @variable(LP, z[1:n, 1:l], Int) # 1 client i visité au temps t, 0 sinon (Attention z[0, t] nb de véhicules 
                                    # qui quittent l'usine au temps t)
    @variable(LP, w[1:n, 1:l], Int) # chargement véhicule avant de faire une livraison au client i au temps t 

    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n) for j in 1:n) for t in 1:l))
    
    indices_array = [i for i in 1:n-1]
    allS = partiesliste(indices_array, 1, n-2)
    
    for t in 1:l # Pour toutes les périodes (l=nb de prériodes)
        sum_d_current_t = sum(sum(p["Clients"].d[ii][tt] for ii in 1:n-1) for tt in t:l)
        Mt = min(p["C"], sum_d_current_t)
        @constraint(LP, prod[t] <= Mt * y[t]) # (4) OK
        @constraint(LP, inv[1, t] <= p["Clients"].L[1]) # (5) OK
        @constraint(LP, z[1, t] <= p["k"]) # (10) OK | Attention k == m sur l'article
        @constraint(LP, 0 <= prod[t]) # (13.1) OK
        @constraint(LP, 0 <= y[t] <= 1) # (14.1) OK
        # Contrainte 16 définie par le Int lors de la déclaration de la variable
        
        
        for ii in allS
            jj = indices_array[indices_array .∉ Ref(ii)]
            # (17) OK :
            @constraint(LP, sum(sum(x[i, j, t] for j in jj) for i in ii) >= sum(q[i, t] for i in ii) / p["Q"])
            if(size(ii)[1] >= 2 && size(ii)[1] <= n-3)
                @constraint(LP, sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    size(ii)[1] - sum(q[i, t] for i in ii) / p["Q"]) # (18) OK
                @constraint(LP, p["Q"] * sum(sum(x[i, j, t] for j in ii) for i in ii) <= 
                    sum(p["Q"] * z[i, t] - q[i, t] for i in ii)) # (19) OK
            end
        end
        
        if(t ≠ 1) 
            @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 1:n) + inv[1, t]) # (2) OK
        end
        for i in 1:n # Parcours de tous les clients
            Mit = min(p["Clients"].L[i], p["Q"], sum_d_current_t)
            xijt_sum = sum(x[i, jj, t] for jj in 1:n)
            if(i ≠ 1) # Nc
                if(t ≠ 1)
                    @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i-1][t] + inv[i, t]) # (3) OK
                    @constraint(LP, inv[i, t-1] + q[i, t] <= p["Clients"].L[i]) # (6) OK
                end
                @constraint(LP, q[i, t] <= Mit * z[i, t]) # (7) OK
                @constraint(LP, xijt_sum == z[i, t]) # (8) OK
                @constraint(LP, 0 <= w[i, t]) # (12.1) OK
                @constraint(LP, w[i, t] <= p["Q"] * z[i, t]) # (12.2) OK
                @constraint(LP, 0 <= z[i, t] <= 1)
            end
            @constraint(LP, xijt_sum + sum(x[jj, i, t] for jj in 1:n) == 2 * z[i, t]) # (9) OK
            @constraint(LP, 0 <= inv[i, t]) # (13.2) OK
            @constraint(LP, 0 <= q[i, t]) # (13.3) OK 
            
            for j in 1:n
                if(i ≠ j)
                    @constraint(LP, w[i, t] - w[j, t] <= q[i, t] - Mit * (1 - x[i, j, t])) # (11) OK
                end
                @constraint(LP, 0 <= x[i, j, t] <= 1) # (14.2) OK
            end
        end
    end
    
    # print(LP)
    # println()

    optimize!(LP)
   
    println(solution_summary(LP, verbose=true))
    
    return 0
end